In [ ]:
!nvidia-smi

In [ ]:
!git clone --branch v0.6 https://github.com/facebookresearch/detectron2.git detectron2_repo
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install zipfile36

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
import os
import cv2

from datetime import datetime
from google.colab.patches import cv2_imshow

from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

from detectron2 import model_zoo
from detectron2.config import get_cfg

from detectron2.engine import DefaultPredictor

from detectron2.engine import DefaultTrainer
from detectron2.projects import point_rend

In [ ]:
!pip install roboflow
from roboflow import Roboflow
API_KEY = "Insert your API Key here"
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("senior-research-project-3n9r5").project("board-polygon-detection")
version = project.version(10)
dataset = version.download("coco-segmentation")

In [ ]:
#Check dataset
DATA_SET_NAME = dataset.name.replace(" ", "-")
print(DATA_SET_NAME)
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

In [ ]:
#To use datasets in Detectron2, you first need to register them as such
# TRAIN SET
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}_train"
TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    TRAIN_DATA_SET_NAME,
    {},
    TRAIN_DATA_SET_ANN_FILE_PATH,
    TRAIN_DATA_SET_IMAGES_DIR_PATH
)

# TEST SET
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}_test"
TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "test")
TEST_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "test", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    TEST_DATA_SET_NAME,
    {},
    TEST_DATA_SET_ANN_FILE_PATH,
    TEST_DATA_SET_IMAGES_DIR_PATH
)

# VALID SET
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}_valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    VALID_DATA_SET_NAME,
    {},
    VALID_DATA_SET_ANN_FILE_PATH,
    VALID_DATA_SET_IMAGES_DIR_PATH
)

In [ ]:
#Confirm the dataset was correctly registered
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

In [ ]:
#Configuring model parameters
# HYPERPARAMETERS
ARCHITECTURE = "pointrend_rcnn_X_101_32x8d_FPN_3x_coco"
CONFIG_FILE_PATH = f"detectron2_repo/projects/PointRend/configs/InstanceSegmentation/{ARCHITECTURE}.yaml"
NUM_CLASSES = 1

# OUTPUT DIR

OUTPUT_DIR_PATH = os.path.join(
    DATA_SET_NAME,
    ARCHITECTURE,
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)


os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

In [ ]:
#Configuring model hyperparameters
cfg = get_cfg()
point_rend.add_pointrend_config(cfg)
cfg.merge_from_file("detectron2_repo/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_X_101_32x8d_FPN_3x_coco.yaml")
cfg.MODEL.WEIGHTS = "model_0094999.pth"
#
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.DEVICE = "cuda"
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 4

cfg.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, True, True, True]  # Enable on res3, res4, res5
cfg.MODEL.RESNETS.DEFORM_MODULATED = True  # Uses flexible receptive fields
cfg.MODEL.RESNETS.DEFORM_NUM_GROUPS = 8  # Improves feature extraction

cfg.SOLVER.BASE_LR = 0.00025  # Lower LR for fine-tuning
cfg.SOLVER.MAX_ITER = 120000  # Train for more iterations
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"  # Gradual learning rate decay
cfg.SOLVER.WARMUP_ITERS = 2000

cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]  # Use all feature levels
cfg.MODEL.FPN.OUT_CHANNELS = 256

cfg.MODEL.POINT_HEAD.NUM_SUBDIVISION_POINTS = 4096  # Higher sampling density
cfg.MODEL.POINT_HEAD.NUM_SUBDIVISION_STEPS = 4

cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES + 1
cfg.MODEL.POINT_HEAD.NUM_CLASSES = NUM_CLASSES + 1
cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

In [ ]:
#Train model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $OUTPUT_DIR_PATH

In [ ]:
# Setup model predictor
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [ ]:
# Run model on test image
TEST_PATH = "<Include your test image path here>"
img = cv2.imread(TEST_PATH)
resized_image = cv2.resize(img, (1280, 1280))
cv2_imshow(resized_image)
outputs = predictor(resized_image)
visualizer = Visualizer(
    resized_image[:, :, ::-1],
    metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
    scale=0.8,
    instance_mode=ColorMode.IMAGE_BW
)
out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])